In [43]:
import pandas as pd
import ast

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 150)

print("done")

done


In [44]:
elist = "../data/editorials-1.jl"
tags = "../data/by_article_110219.jl"

def import_jl(filename):
    filename = filename
    full = open(filename, "r")
    list = []
    for x in full:
        list.append(x)
    return list

editorial_raw = import_jl(elist)
tags_raw = import_jl(tags)

In [45]:
print(editorial_raw[5])
print(tags_raw[5])

{"text": "\nHefty university fees are sending some grad students to food banks", "preview": "\n  \nAs costs rise, students and faculty members are pushing back\r\n  \n", "byline": "<p class=\"byline\">\n  By <a href=\"/author/katie-langin\">Katie Langin</a><time>Aug. 12, 2019</time></p>"}

{"headline": "A warning from the academic underground of adjuncts and contingent faculty", "tags": ["Read more Taken for Granted", "Taken for Granted", "Column", "Non-disciplinary"], "byline": "<p class=\"byline byline--article\">\n  By <a href=\"/author/beryl-lieff-benderly\">Beryl Lieff Benderly</a><time>Jun. 10, 2019 , 2:30 PM</time></p>"}



Next 2 blocks for initially processing jl input files. the blocks are for editorials and tags respectively 
in the final version of this, I want to have them both in the same function
an argument will indicate which file is being processeed. 
I want sidebar to be passed to the function, not inside it--as different scrapes could have different sidebars. However, to do this, I need to know how to make that argument optional, since sidebar will not be necessary for the tags file

In [46]:
def editorial_process(editorial_list):
    editorial_dict = {"headline":[], "preview":[],"authors":[],"date":[] }

    sidebar = ["How to keep a lab notebook",
               "Grad student unions dealt blow as proposed new rule says students aren’t ‘employees’",
               "What do we know about Ph.D. scientists’ career paths?",
               "Three lessons from industry that I’m taking back to academia"
              ]

    #print(editorial_list[1])

    for line in editorial_list:
        if "null" in line:
            line = line.replace("null", "\'null\'")
        line = ast.literal_eval(line.lower())
        if line["text"].replace("\n","")  in sidebar:
            if line["preview"] == "null":
                continue

        editorial_dict["headline"].append(line["text"].replace("\n","").replace("\"",""))
        editorial_dict["preview"].append(line["preview"].replace("\n","").replace("\r","").replace("  ",""))
        num_authors = (len(line["byline"].replace("<",">").split(">"))-5)//4
        au = line["byline"].replace("<",">").split(">")[4]
        for i in range(2,num_authors):
            au += ", " + line["byline"].replace("<",">").split(">")[i*4]
        editorial_dict["authors"].append(au)
        editorial_dict["date"].append(line["byline"].replace("<",">").split(">")[-5].replace(". ","-").replace(", ","-").replace(" ","").replace(",",""))


    editorial_df = pd.DataFrame(editorial_dict)
    editorial_df["date"] = pd.to_datetime(editorial_df.date, format='%b-%d-%Y')
    editorial_df.sort_values(by=["date"], inplace=True)
    
    return [editorial_df, editorial_dict]

editorial = editorial_process(editorial_raw)
editorial_dict_backup = editorial[1]
editorial = editorial[0]
editorial.head()
#editorial_dict_backup

,headline,preview,authors,date
7119,how to write a winning résumé,our columnist explains all you need to know about resumes and cvs,peter fiske,1996-10-18
7115,the commandments of cover letter creation,writing a good cover letter boils down to making a decent presentation of your experience with t...,peter fiske,1996-12-20
7114,his mother cried when he went into sales,"how one man's career change helped him keep his self-respect, make a few bucks, and have a blast...",david g. jensen,1997-05-09
7113,what are headhunters and how do they work?,do you know how the headhunting process works and how it might affect you someday?,david g. jensen,1997-09-12
7112,dressing scientists for success: male case study,"scientists may find shopping for business attire perplexing at first, but this is easily overcom...",peter fiske,1997-09-26


In [47]:
def tags_process(article_tags):
    tags_dict = {"headline":[],"tags":[],"authors":[],"date":[],"time":[] }

    #print(editorial_list[1])

    for line in article_tags:
        if "null" in line:
            line = line.replace("null", "\'null\'")
        line = ast.literal_eval(line.lower())
        head = line["headline"].replace("\n","").replace("\"","")
        tags_dict["headline"].append(head)
        tags = line["tags"]
        unique_tags = []
        for tag in tags:
            if tag not in unique_tags:
                unique_tags.append(tag)
        tags = unique_tags
        if len(tags) < 1:
            tags_dict["tags"].append("[]")
        elif len(tags[0])< 5:
            #tags_dict["tags"].append(", ".join(tags))
            tags_dict["tags"].append(tags)
        elif tags[0][:5] == "Read ":
            if tags[0][10:-1] not in tags[1]:
                included = "false"
                for tag in tags[2:]:
                    if tags[0][10:-1] in tag:
                        #tags_dict["tags"].append(", ".join(tags[1:]))
                        tags_dict["tags"].append(tags)
                        included = "true"
                        break
                if included == "false":
                    tags[0] = tags[0][10:]
                    #tags_dict["tags"].append(", ".join(tags))
                    tags_dict["tags"].append(tags)

            else:
                #tags_dict["tags"].append(", ".join(tags[1:]))
                tags_dict["tags"].append(tags)
        else:
            #tags_dict["tags"].append(", ".join(tags))
            tags_dict["tags"].append(tags)

        num_authors = (len(line["byline"].replace("<",">").split(">"))-5)//4
        au = line["byline"].replace("<",">").split(">")[4]
        for i in range(2,num_authors):
            au += ", " + line["byline"].replace("<",">").split(">")[i*4]
        tags_dict["authors"].append(au)
        date_time = line["byline"].replace("<",">").split(">")[-5]
        time = date_time[-7:]
        date = date_time[:-9].replace(". ","-").replace(", ","-").replace(" ","").replace(",","")
        tags_dict["date"].append(date)
        tags_dict["time"].append(time)


    #print(tags_dict)

    tags_df = pd.DataFrame(tags_dict)
    tags_df.head()
    tags_df["date"] = pd.to_datetime(tags_df.date, format='%b-%d-%Y')
    tags_df["date"] = pd.to_datetime(tags_df["date"])
    tags_df.sort_values(by=["date"], inplace=True)
    return([tags_df, tags_dict])

tags = tags_process(tags_raw)
tags_dict_backup = tags[1]
tags = tags[0]
tags.head()

,headline,tags,authors,date,time
5969,how to write a winning résumé,"[tooling up, advice, graduate, academic, industry, americas, united states]",peter fiske,1996-10-18,8:00 am
5968,the commandments of cover letter creation,"[tooling up, advice, americas]",peter fiske,1996-12-20,0:00 am
5974,his mother cried when he went into sales,"[tooling up, advice, graduate, postdoc, industry, biomedical, americas]",david g. jensen,1997-05-09,8:00 am
5973,what are headhunters and how do they work?,"[tooling up, advice, graduate, midcareer, postdoc, industry, biomedical, americas]",david g. jensen,1997-09-12,8:00 am
5996,dressing scientists for success: male case study,"[tooling up, advice, graduate, postdoc, academic, industry, americas]",peter fiske,1997-09-26,8:00 am


In [48]:
# I wonder why they're different numbers
print(len(editorial))
print(len(tags))
print(len(editorial)-len(tags))

#print(editorial[editorial["headline"] == "Sowing the Seeds of Change"])

# Dammit, I wonder why....

7324
6136
1188


In [35]:
# merge dataframes by headline column

merge_test = editorial.merge(tags, how = "outer", left_on = "headline", right_on = "headline", indicator = True)


print(len(merge_test))
print(len(merge_test[merge_test["_merge"] != "both"]))
merge_test[merge_test["_merge"] != "both"]


6538
421


,headline,preview,authors_x,date_x,tags,authors_y,date_y,time,_merge
41,Graduate Student Enrollments Continue to Decline,null,Nicole Ruediger,1999-01-08,NaN,NaN,NaT,NaN,left_only
62,"Finance 1, The Cash Flow Statement in the Annual Report: Part V of Learnin's From My MBA Series",null,Charles Boulakia,1999-02-05,NaN,NaN,NaT,NaN,left_only
114,Training Scientists How to be Better Teachers: Doing It Yourself,null,Victoria McGovern,1999-04-09,NaN,NaN,NaT,NaN,left_only
115,Training Scientists How to be Better Teachers: Doing It Yourself,null,Victoria McGovern,1999-04-09,NaN,NaN,NaT,NaN,left_only
167,Proteomics: Introduction,null,Donna Morrison,1999-06-04,NaN,NaN,NaT,NaN,left_only
168,Proteomics: Introduction,null,Donna Morrison,1999-06-04,NaN,NaN,NaT,NaN,left_only
192,"The Telephone, Job Search Power Tool",What I want to share with you in this column is how to make better use of the telephone to minim...,David G. Jensen,1999-07-09,NaN,NaN,NaT,NaN,left_only
266,German University Reform: Qualified in Germany Has to Become a Worldwide Symbol for Quality Again,null,Max Huber,1999-10-08,NaN,NaN,NaT,NaN,left_only
267,NSF Criteria,null,Vid Mohan-Ram,1999-10-08,NaN,NaN,NaT,NaN,left_only
373,100 Years of Women at German Universities,null,Sabine Steghaus-Kovac,2000-02-11,NaN,NaN,NaT,NaN,left_only


Generate means of processing sequential dates from the date of the first publication--basically, I hate the python dates function, and I'd rather use a function to go from sequential days to year instead of trying to extract the month and year

In [10]:
# Why am I doing this? I'm trying to calculate the months from the date_seq 
# Right now, this function returns a sequence from the actual start of the publications to the most recent publications
# I would want later versions of this to be more flexible--maybe allowing the start and end times to be set in the function?

def cumulative():
    months_r = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    months_l = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    # I need a list of cumulative days at the end of each month going forward
    cumulative_days = [31, 61, 92]
    cumulative_months = [13]

    for year in range(1997,2021):
        cumulative_months.append(cumulative_months[-1] + 12)
        if year%4 == 0:
            months = months_l
        else:
            months = months_r
        for month in range(1,13):
            #print(year, month)
            #print(months[month-1])
            cumulative_days.append(cumulative_days[-1]+months[month-1])
    return(cumulative_days, cumulative_months)

output = cumulative()

print(cumulative()[0])
print(output[1])

[31, 61, 92, 123, 151, 182, 212, 243, 273, 304, 335, 365, 396, 426, 457, 488, 516, 547, 577, 608, 638, 669, 700, 730, 761, 791, 822, 853, 881, 912, 942, 973, 1003, 1034, 1065, 1095, 1126, 1156, 1187, 1218, 1247, 1278, 1308, 1339, 1369, 1400, 1431, 1461, 1492, 1522, 1553, 1584, 1612, 1643, 1673, 1704, 1734, 1765, 1796, 1826, 1857, 1887, 1918, 1949, 1977, 2008, 2038, 2069, 2099, 2130, 2161, 2191, 2222, 2252, 2283, 2314, 2342, 2373, 2403, 2434, 2464, 2495, 2526, 2556, 2587, 2617, 2648, 2679, 2708, 2739, 2769, 2800, 2830, 2861, 2892, 2922, 2953, 2983, 3014, 3045, 3073, 3104, 3134, 3165, 3195, 3226, 3257, 3287, 3318, 3348, 3379, 3410, 3438, 3469, 3499, 3530, 3560, 3591, 3622, 3652, 3683, 3713, 3744, 3775, 3803, 3834, 3864, 3895, 3925, 3956, 3987, 4017, 4048, 4078, 4109, 4140, 4169, 4200, 4230, 4261, 4291, 4322, 4353, 4383, 4414, 4444, 4475, 4506, 4534, 4565, 4595, 4626, 4656, 4687, 4718, 4748, 4779, 4809, 4840, 4871, 4899, 4930, 4960, 4991, 5021, 5052, 5083, 5113, 5144, 5174, 5205, 5236, 52